In [1]:
#hide
#default_exp vis.gen

# Visualisation Generation

<br>

### Imports

In [2]:
#exports
import json
import pandas as pd

import typer
import croniter
import importlib
from tqdm import tqdm

import matplotlib.pyplot as plt

In [3]:
from IPython.display import JSON

In [4]:
#exports
def rgb_2_plt_tuple(r, g, b):
    """converts a standard rgb set from a 0-255 range to 0-1"""
    
    plt_tuple = tuple([x/255 for x in (r, g, b)])
    
    return plt_tuple

In [23]:
vis_configs = [
    {
        'cron': '0 * * * 1', # midnight every monday
        'function': 'ElexonDataPortal.vis.ei.generate_GB_decarb_progess',
        'kwargs': {
            'dt_col': 'local_datetime',
            'dt_tz': 'Europe/London',
            'url': 'https://api.github.com/repos/AyrtonB/Electric-Insights/git/trees/master?recursive=1',
            'raw_file_prefix': 'https://raw.githubusercontent.com/AyrtonB/Electric-Insights/master/',
            'update_time': pd.Timestamp.now().round('5min').strftime('%Y-%m-%d %H:%M'),
            'dpi': 250,
            'freq': '7D',
            'use_preloaded_ei_df': True,
            'fuel_colour_dict': {
                'Imports & Storage' : rgb_2_plt_tuple(121,68,149), 
                'nuclear' : rgb_2_plt_tuple(77,157,87), 
                'biomass' : rgb_2_plt_tuple(168,125,81), 
                'gas' : rgb_2_plt_tuple(254,156,66), 
                'coal' : rgb_2_plt_tuple(122,122,122), 
                'hydro' : rgb_2_plt_tuple(50,120,196), 
                'wind' : rgb_2_plt_tuple(72,194,227), 
                'solar' : rgb_2_plt_tuple(255,219,65),
            },
            'docs_dir': 'docs',
            'update_time': None,
        }
    },
    {
        'cron': '0 * * * 1', # midnight every monday
        'function': 'ElexonDataPortal.vis.ei.generate_moe',
        'kwargs': {
            'dt_col': 'local_datetime',
            'dt_tz': 'Europe/London',
            'url': 'https://api.github.com/repos/AyrtonB/Electric-Insights/git/trees/master?recursive=1',
            'raw_file_prefix': 'https://raw.githubusercontent.com/AyrtonB/Electric-Insights/master/',
            'reg_dates_start': '2010-01-01',
            'reg_dates_end': None,
            'reg_dates_freq': '13W',
            'num_fits': 15, 
            'x_pred': None,
            'dt_idx': None,
            'dpi': 250, 
            'use_preloaded_ei_df': True,
            'img_name': 'moe_surface',
            'docs_dir': 'docs',
            'update_time': None,
        }
    },
    {
        'cron': '0,30 * * * *', # every half-hour
        'function': 'ElexonDataPortal.vis.lolp.generate_lolpdrm_imgs_text',
        'kwargs': {
            'api_key': None,
            'fcst_horizons': [8, 4, 2, 1],
            'update_time': None,
            'docs_dir': 'docs',
            'update_time': None,
        }
    },
    {
        'cron': '0,30 * * * *', # every half-hour
        'function': 'ElexonDataPortal.vis.map.generate_map',
        'kwargs': {
            'data_dir': 'data/PN',
            'api_key': None,
            'update_time': None,
            'powerdict_url': 'https://raw.githubusercontent.com/OSUKED/Power-Station-Dictionary/main/data/output/power_stations.csv',
            'js_template_fp': 'templates/map.js',
            'js_docs_fp': 'docs/js/map.js', 
            'md_template_fp': 'templates/map.md', 
            'plants_geojson_fp': 'data/power_plants.json',
            'plants_geojson_url': 'https://raw.githubusercontent.com/OSUKED/ElexonDataPortal/master/data/power_plants.json',
            'routes_geojson_url': 'https://raw.githubusercontent.com/OSUKED/ElexonDataPortal/master/data/network_routes.json'

        }
    }
]

JSON(vis_configs)

<IPython.core.display.JSON object>

In [24]:
save_vis_configs = False

if save_vis_configs == True:
    with open('../data/vis_configs.json', 'w') as f:
        json.dump(vis_configs, f)

In [7]:
#exports
def get_vis_func(func_path):
    *lib_path, func_name = func_path.split('.')
    lib_obj = importlib.import_module('.'.join(lib_path))
    
    func = getattr(lib_obj, func_name)

    return func

def get_vis_md_text(vis_config, docs_dir=None, update_time=None):
    func_path = vis_config['function']
    kwargs = vis_config['kwargs']

    if (docs_dir is not None) and ('docs_dir' in kwargs.keys()):
        kwargs['docs_dir'] = docs_dir

    if (update_time is not None) and ('update_time' in kwargs.keys()):
        kwargs['update_time'] = update_time

    vis_func = get_vis_func(func_path)
    vis_md_text = vis_func(**kwargs)
    plt.close()
    
    return vis_md_text

In [8]:
docs_dir = '../docs'

vis_config = vis_configs[0]
vis_md_text = get_vis_md_text(vis_config, docs_dir=docs_dir)

print(vis_md_text)

### Measuring the Progress and Impacts of Decarbonising British Electricity

The figures shown here are attempts to replicate the visualisations from [this paper](https://www.sciencedirect.com/science/article/pii/S0301421516307017) by Dr Iain Staffell which finds that:

* CO2 emissions from British electricity have fallen 46% in the three years to June 2016.
* Emissions from imports and biomass are not attributed to electricity, but add 5%.
* Coal capacity fell 50% and output 75% due to prices, competition and legislation.
* Wind, solar and biomass provided 20% of demand in 2015, with a peak of 45%.
* Prices have become more volatile and net demand is falling towards must-run nuclear.

These figures will be updated on a weekly basis, the last update was at: None

<br>

#### Weekly Averaged Generation Mix

The following figure shows a stacked plot of the generation from different fuel types over time, averaged on a weekly basis. The original plot can be found [here](https://www.scienced

In [9]:
#exports
def get_rerun_vis_bool(vis_config):
    if 'last_update_time' not in vis_config.keys():
        return True
    else:
        last_update_time = pd.to_datetime(vis_config['last_update_time']).tz_localize('Europe/London')
        
    cron = croniter.croniter(vis_config['cron'], pd.Timestamp.now()-pd.Timedelta(weeks=1))
    cron_dts = pd.to_datetime([cron.get_next() for i in range(10*48*7)], unit='s').tz_localize('UTC').tz_convert('Europe/London')
    
    s_cron_dts_time_delta_to_now = pd.Series((cron_dts - pd.Timestamp.now(tz='Europe/London')).total_seconds())
    assert (s_cron_dts_time_delta_to_now<0).sum()>0 and (s_cron_dts_time_delta_to_now>0).sum()>0, 'The cron dts being assessed do not cross the current time'
    
    s_cron_dts_time_delta_to_last_update_time = pd.Series((cron_dts - last_update_time).total_seconds())
    if s_cron_dts_time_delta_to_now.abs().idxmin() == s_cron_dts_time_delta_to_last_update_time.abs().idxmin():
        return False
    
    avg_adj_dt_time_delta_s = pd.Series(cron_dts).diff(1).dropna().dt.total_seconds().mean()
    min_time_delta_s = s_cron_dts_time_delta_to_now.abs().min()

    rerun_vis = avg_adj_dt_time_delta_s >= min_time_delta_s

    return rerun_vis

In [10]:
rerun_vis = get_rerun_vis_bool(vis_config)

rerun_vis

True

In [11]:
#exports
def update_vis_configs(
    vis_configs, 
    docs_dir: str='docs',
    override_rerun_vis_bool: bool=False
):
    for i, vis_config in enumerate(vis_configs):
        update_time = pd.Timestamp.now().round('5min').strftime('%Y-%m-%d %H:%M')
        rerun_vis = get_rerun_vis_bool(vis_config)
        
        if override_rerun_vis_bool == True:
            rerun_vis = True

        if rerun_vis == True:
            vis_md_text = get_vis_md_text(vis_config, docs_dir=docs_dir, update_time=update_time)
            vis_configs[i]['md_text'] = vis_md_text
            vis_configs[i]['last_update_time'] = update_time
            
    return vis_configs

In [12]:
docs_dir = '../docs'
data_dir = '../data'

with open(f'{data_dir}/vis_configs.json', 'r') as f:
    vis_configs = json.load(f)

vis_configs = update_vis_configs(vis_configs, docs_dir=docs_dir)
    
with open(f'{data_dir}/vis_configs.json', 'w') as f:
    json.dump(vis_configs, f)

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [05:33<00:00,  7.10s/it]
C:\Users\Ayrto\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [13]:
# add in a mini example func template

In [14]:
all_vis_md_texts = ['# Visualisations'] + [vis_config['md_text'] for vis_config in vis_configs]
combined_md_text = '\n\n<br>\n\n'.join(all_vis_md_texts)

print(combined_md_text)

# Visualisations

<br>

### Measuring the Progress and Impacts of Decarbonising British Electricity

The figures shown here are attempts to replicate the visualisations from [this paper](https://www.sciencedirect.com/science/article/pii/S0301421516307017) by Dr Iain Staffell which finds that:

* CO2 emissions from British electricity have fallen 46% in the three years to June 2016.
* Emissions from imports and biomass are not attributed to electricity, but add 5%.
* Coal capacity fell 50% and output 75% due to prices, competition and legislation.
* Wind, solar and biomass provided 20% of demand in 2015, with a peak of 45%.
* Prices have become more volatile and net demand is falling towards must-run nuclear.

These figures will be updated on a weekly basis, the last update was at: 2021-07-05 13:15

<br>

#### Weekly Averaged Generation Mix

The following figure shows a stacked plot of the generation from different fuel types over time, averaged on a weekly basis. The original plot can 

In [15]:
with open('../docs/visualisations.md', 'w', encoding='utf-8') as f:
    f.write(combined_md_text)

<br>

`python -m ElexonDataPortal.vis.gen`

In [19]:
#exports
app = typer.Typer()

@app.command()
def update_vis(
    docs_dir: str='docs',
    data_dir: str='data',
    override_rerun_vis_bool: bool=False
):
    with open(f'{data_dir}/vis_configs.json', 'r') as f:
        vis_configs = json.load(f)

    vis_configs = update_vis_configs(vis_configs, docs_dir=docs_dir, override_rerun_vis_bool=override_rerun_vis_bool)

    with open(f'{data_dir}/vis_configs.json', 'w') as f:
        json.dump(vis_configs, f)
        
    prefix_text = """# Visualisations

On this page you can view visualisations of key phenomena in the GB power sector, ranging from long-term trends in the generation-mix and market prices to information on excess capacity in the grid. All data used in these visualisations was either sourced directly from BMRS using the `ElexonDataPortal` client, or has been derived from BMRS data streams. As with the other components of the `ElexonDataPortal` the code to generate these visualisations is open-source and users are welcome to contribute their own visualisations, for more detail on how to do this please refer to the [user contribution guide](#contributor-guide)
    """
        
    suffix_text = """### Contributor Guide

We encourage users to contribute their own visualisations which the `ElexonDataPortal` will then update automatically. To this end the library adopts a standardised format for generating visualisations, the core component of which is the `data/vis_configs.json` file to which you will have to add detail on your visualisation function: 

```javascript
[
    ...
    {
        "cron": "0 * * * *", # the update schedule, in this instance to run at midnight every sunday
        "function": "path_to_function", # e.g. ElexonDataPortal.vis.generate_vis
        "kwargs": {
            'api_key': null,  # if no api_key is passed then the client will try and look for the `BMRS_API_KEY` environment variable
            'update_time': null, # if no update_time is passed you should generate it yourself, e.g. with `pd.Timestamp.now().round('5min').strftime('%Y-%m-%d %H:%M')`
            'docs_dir': 'docs', # in almost all circumstances this should just be `docs`
            "optional_kwarg": "optional_value" # you can specify any additional keyword arguments that your function requires
        }
    },
    ...
]
```

<br>

The other core component is writing the function that generates the visualisation. This function should require parameters for the `docs_dir`, `api_key`, and `update_time` but can include optional parameters that you wish to specify, it should then return markdown text which will be used to populate the *Visualisations* page. These functions will normally contain three steps: data retrieval, generating the visualisation, and generating the accompanying text - an example can be seen below.

```python
import pandas as pd
import matplotlib.pyplot as plt
from ElexonDataPortal.api import Client

def generate_vis(
    docs_dir: str='docs',
    api_key: str=None,
    update_time: str=pd.Timestamp.now().round('5min').strftime('%Y-%m-%d %H:%M'),
) -> str:

    # Data Retrieval
    client = Client(api_key=api_key)
    df = client.get_data_stream(param1, param2)
    
    # Generating the Visualisation
    fig, ax = plt.subplots(dpi=150)
    df.plot(ax=ax)
    fig.savefig(f'{docs_dir}/img/vis/great_vis_name.png')
    
    # Generating the Text
    md_text = f\"\"\"### Title
    
Explanation of what your visualisation shows

![](img/vis/great_vis_name.png)
\"\"\"
    
    return md_text
```

N.b. the path to the image should be relative to the `docs` directory.

If you require any assistance in this process please start a discussion [here](https://github.com/OSUKED/ElexonDataPortal/discussions) and we'll endeavour to help as best we can.
"""
        
    all_vis_md_texts = [prefix_text] + [vis_config['md_text'] for vis_config in vis_configs] + [suffix_text]
    combined_md_text = '\n\n<br>\n\n'.join(all_vis_md_texts)

    with open(f'{docs_dir}/visualisations.md', 'w', encoding='utf-8') as f:
        f.write(combined_md_text)
        
    return

if __name__ == '__main__' and '__file__' in globals():
    app()

In [22]:
update_vis(docs_dir='../docs', data_dir='../data', override_rerun_vis_bool=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [05:40<00:00,  7.25s/it]
C:\Users\Ayrto\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [ ]:
# need to write a func that will be run by the GH action 
# should check the time between now and any cron jobs descs, then run those that are within 24hrs
# no cron jobs will run more frequently than every 24hrs

# How to contribute?
# * Create a function that writes an image to the `docs/img/vis` directory 
# * Ensure that same function returns a markdown string which will render the desired text and images if loaded from the `docs` directory
# * Add the function name, schedule (in cron notation), and any kwargs to be passed to the function as a new item in the `data/vis_configs.json` file

In [2]:
#hide
from ElexonDataPortal.dev.nbdev import notebook2script
notebook2script('vis-00-gen.ipynb')

Converted vis-00-gen.ipynb.
